In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE84632"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE84632"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE84632.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84632.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE84632.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of peripheral blood mononuclear cells from adults with sickle cell disease (UIC cohort)"
!Series_summary	"Sickle cell disease is associated with systemic complications, many associated with either severity of disease or increased risk of mortality. We sought to identify a circulating gene expression profile whose predictive capacity spanned the spectrum of these poor outcomes in sickle cell disease."
!Series_summary	"The Training cohort consisted of patients with SCD who were prospectively recruited from the University of Illinois. The Testing cohort consisted of a combination of patients prospectively seen at two separate institutions including the University of Chicago and Howard University"
!Series_overall_design	"The gene expression of PBMC from 172 sickle cell disease patients at UIC were analyzed"
Sample Characteristics Dictionary:
{0: ['tissue: peripheral blood'], 1: ['cell type: mononuclear cells'], 2: ['disease: Sickle cel

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset likely contains gene expression data 
# as it specifically mentions "Gene expression of peripheral blood mononuclear cells"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Sickle_Cell_Anemia), we can see from sample characteristics that all patients have SCD
# The key 2 contains "disease: Sickle cell disease" but since all patients have the same disease,
# there's no variation to study
trait_row = None

# For age, there is no information in the sample characteristics
age_row = None

# For gender, there is no information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since trait_row is None (all patients have the same disease), we don't need to define convert_trait
# But for completeness, we'll define it anyway
def convert_trait(value):
    if value is None:
        return None
    
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "sickle cell disease" in value or "scd" in value:
        return 1
    else:
        return 0

# Define conversion functions for age and gender even though we don't have that data
def convert_age(value):
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = value.lower().strip()
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Since trait_row is None, we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 65
Header line: "ID_REF"	"GSM2243130"	"GSM2243131"	"GSM2243132"	"GSM2243133"	"GSM2243134"	"GSM2243135"	"GSM2243136"	"GSM2243137"	"GSM2243138"	"GSM2243139"	"GSM2243140"	"GSM2243141"	"GSM2243142"	"GSM2243143"	"GSM2243144"	"GSM2243145"	"GSM2243146"	"GSM2243147"	"GSM2243148"	"GSM2243149"	"GSM2243150"	"GSM2243151"	"GSM2243152"	"GSM2243153"	"GSM2243154"	"GSM2243155"	"GSM2243156"	"GSM2243157"	"GSM2243158"	"GSM2243159"	"GSM2243160"	"GSM2243161"	"GSM2243162"	"GSM2243163"	"GSM2243164"	"GSM2243165"	"GSM2243166"	"GSM2243167"	"GSM2243168"	"GSM2243169"	"GSM2243170"	"GSM2243171"	"GSM2243172"	"GSM2243173"	"GSM2243174"	"GSM2243175"	"GSM2243176"	"GSM2243177"	"GSM2243178"	"GSM2243179"	"GSM2243180"	"GSM2243181"	"GSM2243182"	"GSM2243183"	"GSM2243184"	"GSM2243185"	"GSM2243186"	"GSM2243187"	"GSM2243188"	"GSM2243189"	"GSM2243190"	"GSM2243191"	"GSM2243192"	"GSM2243193"	"GSM2243194"	"GSM2243195"	"GSM2243196"	"GSM2243197"	"GSM2243198"	"GSM2243199"	"GSM2243200"	"GSM2243201"	"GSM2243202"	

Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Let's analyze the gene identifiers in the gene expression data
# Looking at the identifiers: "16650001", "16650003", "16650005", etc.

# These appear to be numeric IDs rather than human gene symbols
# Human gene symbols typically have alphabetic characters like "BRCA1" or "TP53"
# These numeric IDs appear to be probe or feature identifiers from a microarray platform
# They will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE84632
Line 6: !Series_title = Gene expression of peripheral blood mononuclear cells from adults with sickle cell disease (UIC cohort)
Line 7: !Series_geo_accession = GSE84632
Line 8: !Series_status = Public on Sep 08 2017
Line 9: !Series_submission_date = Jul 20 2016
Line 10: !Series_last_update_date = Mar 15 2019
Line 11: !Series_pubmed_id = 28373264
Line 12: !Series_summary = Sickle cell disease is associated with systemic complications, many associated with either severity of disease or increased risk of mortality. We sought to identify a circulating gene expression profile whose predictive capacity spanned the spectrum of these poor outcomes in sickle cell disease.
Line 13: !Series_summary = The Trai

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for mapping
# Looking at the gene annotation preview, we can see that 'ID' corresponds to the same 
# probe ID format as in the gene expression data.

# First let's get the gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)
print("Gene annotation columns:", gene_annotation.columns.tolist())

# Check the columns to understand what kind of annotation we have
for col in gene_annotation.columns:
    non_null_values = gene_annotation[col].dropna().head(3).tolist()
    if non_null_values:
        print(f"{col} sample values: {non_null_values}")

# Since there's no clear gene symbol column, we need a different approach
# Let's modify our extract_human_gene_symbols function to also accept RefSeq IDs
def custom_extract_gene_info(text):
    """Custom function to extract gene information from annotation text including RefSeq IDs"""
    if not isinstance(text, str):
        return None
    
    # Accept RefSeq IDs for this dataset since that's what we have
    if isinstance(text, str) and text.startswith(('NR_', 'XR_', 'NM_', 'XM_')):
        return text
    
    return None

# Use a direct mapping from probe IDs to the GB_ACC column which contains RefSeq IDs
# Create a simple mapping dataframe with ID and GB_ACC columns
mapping_df = gene_annotation[['ID', 'GB_ACC']].copy()
mapping_df = mapping_df.dropna()
mapping_df = mapping_df.rename(columns={'GB_ACC': 'Gene'})
print("\nGene mapping dataframe preview:")
print(preview_df(mapping_df))

# Count how many valid mappings we have
print(f"Number of probes with gene mappings: {len(mapping_df)}")

# 3. Convert probe-level measurements to gene expression data
# We'll customize the mapping approach since we're working with RefSeq IDs
# instead of standard gene symbols

# First ensure our gene_data has the right index type
gene_data.index = gene_data.index.astype(str)

# Create a mapping dictionary from probe ID to RefSeq ID
id_to_gene = dict(zip(mapping_df['ID'], mapping_df['Gene']))

# Apply a direct mapping - simpler than the extract_human_gene_symbols approach
# since we're working with RefSeq IDs directly
gene_values = {}

# For each probe in our expression data
for probe_id in gene_data.index:
    if probe_id in id_to_gene:
        gene = id_to_gene[probe_id]
        # If this is the first time we've seen this gene, initialize its values
        if gene not in gene_values:
            gene_values[gene] = gene_data.loc[probe_id].values
        else:
            # Otherwise, add the probe's values to the gene's existing values
            gene_values[gene] = gene_values[gene] + gene_data.loc[probe_id].values

# Convert our dictionary to a DataFrame
gene_data_mapped = pd.DataFrame(gene_values, index=gene_data.columns).T

# 5. Preview the resulting gene expression data
print("\nGene expression data preview (after mapping):")
print(f"Shape: {gene_data_mapped.shape}")
if len(gene_data_mapped) > 0:
    print(f"First 5 gene symbols: {gene_data_mapped.index[:5].tolist()}")
else:
    print("No genes were mapped. This dataset may not have proper gene annotations.")

# 6. Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_mapped.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")

# In case the mapping didn't work and we got an empty dataframe, 
# let's provide a fallback - save the original probe-level data
if len(gene_data_mapped) == 0:
    print("\nWARNING: No genes were mapped. Saving probe-level data as a fallback.")
    gene_data.to_csv(out_gene_data_file)
    print(f"Probe-level data saved to: {out_gene_data_file}")
    
    # Flag that no proper gene mapping was available for this dataset
    print("\nThis dataset lacks proper gene symbol annotations in the provided files.")
    is_gene_available = False
    
    # Update metadata
    is_trait_available = False  # Already determined to be False in Step 2
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=is_gene_available, 
        is_trait_available=is_trait_available,
        is_biased=None,
        df=pd.DataFrame(),
        note="Dataset lacks proper gene symbol annotations and trait information."
    )
else:
    print(f"Successfully mapped {len(gene_data_mapped)} genes.")


Gene annotation columns: ['ID', 'RANGE_STRAND', 'RANGE_START', 'RANGE_END', 'total_probes', 'GB_ACC', 'SPOT_ID', 'RANGE_GB']
ID sample values: ['16657436', '16657440', '16657445']


RANGE_STRAND sample values: ['+', '+', '+']
RANGE_START sample values: [12190.0, 29554.0, 69091.0]
RANGE_END sample values: [13639.0, 31109.0, 70008.0]
total_probes sample values: [25.0, 28.0, 8.0]
GB_ACC sample values: ['NR_046018', 'NR_024368', 'NR_029406']


SPOT_ID sample values: ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008']
RANGE_GB sample values: ['NC_000001.10', 'NC_000001.10', 'NC_000001.10']



Gene mapping dataframe preview:
{'ID': ['16657436', '16657450', '16657476', '16657485', '16657514'], 'Gene': ['NR_046018', 'NR_024368', 'NR_029406', 'XR_132471', 'NR_047526']}
Number of probes with gene mappings: 17623



Gene expression data preview (after mapping):
Shape: (16346, 172)
First 5 gene symbols: ['NR_046018', 'NR_024368', 'NR_029406', 'XR_132471', 'NR_047526']



Gene expression data saved to: ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84632.csv
Successfully mapped 16346 genes.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Get the most recently created gene data from previous step
# Instead of trying to normalize with NCBI gene symbols (which doesn't work for RefSeq IDs),
# we'll use the gene_data_mapped from the previous step
print(f"Gene data shape before processing: {gene_data_mapped.shape}")

# We can't normalize RefSeq IDs with the standard gene symbol dictionary
# So we'll skip the normalization step and work with the RefSeq IDs directly
print("Using RefSeq IDs as gene identifiers since they cannot be normalized to standard gene symbols.")
normalized_gene_data = gene_data_mapped

# Save the mapped gene data (using RefSeq IDs)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data with RefSeq IDs saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data with RefSeq IDs but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains only {trait} patients with no healthy controls, making it unsuitable for case-control analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before processing: (16346, 172)
Using RefSeq IDs as gene identifiers since they cannot be normalized to standard gene symbols.


Gene data with RefSeq IDs saved to ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84632.csv
Trait row is None. Cannot extract trait information from clinical data.
Abnormality detected in the cohort: GSE84632. Preprocessing failed.
Data was determined to be unusable due to missing trait indicators and was not saved
